In [ ]:
# File system manangement
import os
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 
%config InlineBackend.figure_format = 'retina' ## This is preferable for retina display. 
# matplotlib and seaborn for plotting
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="darkgrid")
import pandas as pd 
import numpy as np

In [ ]:
# List files available
print(os.listdir("../input/"))

In [ ]:
# Read Customer usage file
Customer_usage = pd.read_csv('../input/Customer_usage.txt',delimiter='\t', decimal=',')
print('Customer usage shape: ', Customer_usage.shape)
Customer_usage.head()

In [ ]:
# Read Customer churn file
Customer_churn = pd.read_csv('../input/Customer_churn.txt',delimiter='\t')
print('Customer churn shape: ', Customer_churn.shape)
Customer_churn.head()

In [ ]:
# Examining missing values in customer usage
total = Customer_usage.isnull().sum().sort_values(ascending=False)
percent = round((Customer_usage.isnull().sum().sort_values(ascending=False)/Customer_usage.shape[0])*100,2)
Missing_Values =pd.concat([total,percent], axis = 1,keys= ['Missing Values', '% of Total Values'])
Missing_Values.head(10)

In [ ]:
# Examining missing Value in customer churn
total = Customer_churn.isnull().sum().sort_values(ascending=False)
percent = round((Customer_churn.isnull().sum().sort_values(ascending=False)/Customer_churn.shape[0])*100,2)
Missing_Values =pd.concat([total,percent], axis = 1,keys= ['Missing Values', '% of Total Values'])
Missing_Values

In [ ]:
# Number of each type of column
Customer_usage.dtypes.value_counts()

## Descriptive statistics

In [ ]:
# Define a fuction to set the bar width
widthbars  = [0.3, 0.3, 0.3]
def set_bar_width(ax):
    for p, newwidth in zip(ax.patches, widthbars):
        x = p.get_x()
        width = p.get_width()
        centre = x + width/2.
        p.set_x(centre - newwidth/2.)
        p.set_width(newwidth)

# Define a function to show values on bar
def show_value_on_bar(ax):        
    for p in ax.patches:
        x = p.get_x() + p.get_width() / 2
        y = p.get_y() + p.get_height()
        value = '{:.2f}'.format(p.get_height())
        ax.text(x, y, value, color='black',fontsize=15,ha="center")
    


In [ ]:
# Compute totals per month/year on all numeic features except id columns and user_lifetime

df_stat= Customer_usage.drop(columns = ['user_account_id','user_intake'])
df_stat_sum_per_period = df_stat.drop(columns = ['user_lifetime']).groupby(['year','month'], as_index = False).agg(['sum']).reset_index()
df_stat_sum_per_period['period']=df_stat_sum_per_period['month'].astype(str)+'/'+df_stat_sum_per_period['year'].astype(str)
df_stat_sum_per_period.head()

In [ ]:
# barplots

import matplotlib.pyplot as plt
i=0
plt.figure(figsize=(25,120))
for col in df_stat_sum_per_period.iloc[:,2:-1]:
    if col[0] != 'user_lifetime':
        plt.subplot(20,3,i+1)
        ax= sns.barplot(x='period',y= col ,data=df_stat_sum_per_period,palette="rocket",linewidth= 2  )
        plt.title("Total of "+ col[0] + " per period", fontsize = 15)
        plt.ylabel("Total of "+ col[0], fontsize = 10)
        plt.xlabel("Period",fontsize = 10)
        set_bar_width(ax)
        show_value_on_bar(ax)       
        i=i+1

In [ ]:
# Compute % of customer per month on the 4 binary columns
i=0
plt.figure(figsize=(20,10))
for col in Customer_usage.iloc[:,8:12]:
    plt.subplot(2,2,i+1)
    ax= sns.barplot(x='month',y= col ,data=Customer_usage,palette="rocket",linewidth=2 )
    plt.title("% of "+ col + " per month", fontsize = 15)
    plt.ylabel("% of "+ col, fontsize = 12)
    plt.xlabel("Month",fontsize = 12)
    set_bar_width(ax)
    show_value_on_bar(ax)       
    i=i+1

In [ ]:
# Compute totals over the whole dataset except id columns and user_lifetime

df_stat_sum = df_stat.drop(columns = ['user_lifetime','year','month']).agg(['sum'])
df_stat_sum.head()


In [ ]:
df_stat.drop(columns = ['user_lifetime','year','month']).describe()

In [ ]:
# columns distribution
import matplotlib.pyplot as plt
i=0
plt.figure(figsize=(25,120))
for col in df_stat.iloc[:,2:]:
    plt.subplot(21,3,i+1)
    plt.hist(df_stat[col],bins=25)
    plt.title(col, fontsize = 20)
    i=i+1

## Churn Prediction

In [ ]:
#Preparing the dataset for the churn prediction model (one row per cusstomer)
# Group by the client id, calculate aggregation statistics
customer_usage_agg_by_id= Customer_usage.drop(columns = ['year','month','user_intake']).groupby('user_account_id', as_index = False).agg(['mean']).reset_index()
customer_usage_agg_by_id.head()

In [ ]:
customer_usage_agg_by_id.shape

In [ ]:
# List of column names
columns = ['user_account_id']

# Iterate through the variables names
for var in customer_usage_agg_by_id.columns.levels[0]:
    # Skip the id name
    if var != 'user_account_id':
        
        # Iterate through the stat names
        for stat in customer_usage_agg_by_id.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('%s_%s' % (stat, var))


In [ ]:
# Assign the list of columns names as the dataframe column names
customer_usage_agg_by_id.columns = columns
customer_usage_agg_by_id.head()

In [ ]:
# Consider max user_lifetime by customer instead of the mean
max_userlifetime_by_id= Customer_usage[['user_account_id','user_lifetime']].groupby('user_account_id', as_index = False).agg(['max']).reset_index()
max_userlifetime_by_id.columns =['user_account_id','max_user_lifetime']
customer_usage_agg_by_id = customer_usage_agg_by_id.merge(max_userlifetime_by_id, on = 'user_account_id', how = 'inner')
customer_usage_agg_by_id.drop(columns = ['mean_user_lifetime'], inplace =True)
customer_usage_agg_by_id.head()


In [ ]:
# Merge with the target column
data_by_customer = customer_usage_agg_by_id.merge(Customer_churn, on = 'user_account_id', how = 'left')
data_by_customer.head()

In [ ]:
data_by_customer.drop(columns = ['year','month','user_account_id'],inplace =True)
data_by_customer.head()

In [ ]:
data_by_customer.shape

In [ ]:
features = data_by_customer.iloc[:,:-1]
target = data_by_customer['churn']

## Exploratory Data Analysis

In [ ]:
# Examining the distribution of the target value
f,ax=plt.subplots(1,2,figsize=(18,8))
data_by_customer['churn'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)

ax[0].set_title('Churn')
ax[0].set_ylabel('')
sns.countplot('churn',data = data_by_customer,ax=ax[1])  
show_value_on_bar(ax[1])    
ax[1].set_title('Churn')
plt.show()

From this information, we see this is an imbalanced classification problem. There are far more customers that didn't churn than customers that churn. 

In [ ]:
features.describe()

In [ ]:
# columns distribution
i=0
plt.figure(figsize=(25,120))
for col in features:
    plt.subplot(21,3,i+1)
    plt.hist(features[col],bins=25)
    plt.title(col, fontsize = 20)
    i=i+1

In [ ]:
# compute correlations among features (heatmap)
mask = np.zeros_like(features.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

plt.subplots(figsize = (15,12))
sns.heatmap(features.corr(), 
            annot=False,
            mask = mask,
            cmap = 'RdBu_r',# 'YlGnBu',
            linewidths=0.1, 
            linecolor='white',
            vmax = .9,
            square=True)
plt.title("Correlations Among Features", y = 1.03,fontsize = 20);

In [ ]:
# compute correlatons with the target column
correlations = data_by_customer.corr()['churn'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

In [ ]:
# kde plots: Distribution of the most correlated features to the target

temp_list= list(correlations.abs().sort_values(ascending=True).tail(12).index)
temp_list.remove('churn')
most_corr = data_by_customer.loc[:, temp_list]
# temp.head()

i=0
plt.figure(figsize=(25,25))
for col in most_corr:
    plt.subplot(4,3,i+1)
    sns.kdeplot(data_by_customer.loc[data_by_customer['churn'] == 0,col] ,color='gray',shade=True, label = 'churn == 0')
    sns.kdeplot(data_by_customer.loc[data_by_customer['churn'] == 1,col] ,color='g',shade=True, label = 'churn == 1')
    plt.title(col+ ' Distribution', fontsize = 15)
    i=i+1

In [ ]:
# Define Catplot function 
def cat_plot(feature, cut=12):
    temp = data_by_customer.loc[:, [feature, "churn"]]
    temp[feature + "_binned"] = pd.qcut(temp[feature], cut, duplicates="drop")
    ax = sns.catplot(
        x="churn",
        y=feature + "_binned",
        data=temp,
        kind="bar",
        height=5,
        aspect=2.7,
    )
    
    plt.xlabel('Churn rate')

In [ ]:
# cat plot of the most correlated features to the target
for col in most_corr: 
    cat_plot(col, cut=9)
   

## Downsampling

In [ ]:
# from sklearn.utils import resample
# # Class count
# count_majority, count_minorty = data_by_customer.target.value_counts()
# # Divide by class
# df_majority = data_by_customer[data_by_customer['churn'] == 0]
# df_minority  = data_by_customer[data_by_customer['churn'] == 1]
# 
# df_majority_downsampled = resample(df_majority, 
#                                  replace=False,    # sample without replacement
#                                  n_samples=count_minorty,     # to match minority class
#                                  random_state=123)
# df_downsampled = pd.concat([df_majority_downsampled, df_minority])
# print('Random down-sampling:')
# print(df_downsampled.churn.value_counts())
# 
# df_downsampled.churn.value_counts().plot(kind='bar', title='Count (churn)');
# features = df_downsampled.iloc[:,:-1]
# target = df_downsampled['churn']

In [ ]:
# Splitting data into train set and test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(features,target,test_size=0.2,stratify=target)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
# Scaling Data
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler().fit(X_train)
X_train_Scaled = Scaler.transform(X_train)
X_test_Scaled = Scaler.transform(X_test)


## Prediction Models: Training and Evaluation 

In [ ]:
# Define a roc curve plot function
from sklearn.metrics import roc_curve,auc
def Plot_Roc_Curve(label):
    fpr,tpr, thres = roc_curve(y_test,y_pred)
    fpr_train,tpr_train, thres = roc_curve(y_train,y_pred_train)
    roc_auc= auc(fpr,tpr)
    roc_auc_train= auc(fpr_train,tpr_train)
    plt.figure()
    plt.plot(fpr, tpr, label= 'ROC curve on testing set (area = %0.2f)' % roc_auc, linewidth= 4)
    plt.plot(fpr_train, tpr_train, label= 'ROC curve on traing set (area = %0.2f)' % roc_auc_train, linewidth= 4)
    plt.plot([0,1],[0,1], 'k--', linewidth = 4)
    plt.xlim([0.0,1.0])
    plt.ylim([0.0,1.05])
    plt.xlabel('False Positive Rate', fontsize = 18)
    plt.ylabel('True Positive Rate', fontsize = 18)
    plt.title('ROC curve/'+ label, fontsize= 18)
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Define a precision recall plot function
from sklearn.metrics import precision_recall_curve
def Plot_Precision_Recall_Curve(label):
    precision, recall, _ = precision_recall_curve(y_test, y_pred)
    precision_train, recall_train, _ = precision_recall_curve(y_train, y_pred_train)
    PR_AUC = auc(recall, precision)
    PR_AUC_train = auc(recall_train, precision_train)
    plt.figure()
    plt.plot(recall, precision, label='PR curve on test set (area = %0.2f)' % PR_AUC, linewidth=4)
    plt.plot(recall_train, precision_train, label='PR curve on train set (area = %0.2f)' % PR_AUC_train, linewidth=4)
    plt.xlabel('Recall', fontsize=18)
    plt.ylabel('Precision', fontsize=18)
    plt.title('Precision Recall Curve/'+label,fontsize=18)
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
def plot_feature_importances(df):
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:20]))), 
            df['importance_normalized'].head(20), 
            align = 'center', edgecolor = 'k')
  
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:20]))))
    ax.set_yticklabels(df['feature'].head(20))
   
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
   
    return df

## Gaussian Naive Bayes

In [ ]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB()

GNB.fit(X_train_Scaled, y_train)


y_pred= GNB.predict_proba(X_test_Scaled)[:, 1]
y_pred_train= GNB.predict_proba(X_train_Scaled)[:, 1]




In [ ]:
Plot_Roc_Curve('Gaussian Naive Bayes')
Plot_Precision_Recall_Curve('Gaussian Naive Bayes')

##  Logistic Regression

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
LG = LogisticRegression()

params = {'C': np.logspace(-3,3,10),'penalty':['l1','l2'] }
LG_CV = RandomizedSearchCV(LG,params,n_iter=20,cv=5,verbose = False,scoring='roc_auc', n_jobs=-1)
LG_CV.fit(X_train_Scaled, y_train)
print('Best params: ')
print(LG_CV.best_params_) 
print('Best score: ')
print(LG_CV.best_score_)

In [ ]:
y_pred= LG_CV.predict_proba(X_test_Scaled)[:, 1]
y_pred_train= LG_CV.predict_proba(X_train_Scaled)[:, 1]


In [ ]:
Plot_Roc_Curve('Logistic Regression')
Plot_Precision_Recall_Curve('Logistic Regression')

##  K-Nearest Neighbor classifier(KNN)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(metric='minkowski', p=2)
k_range = range(1,31)
weights_options=['uniform','distance']
params = {'n_neighbors':k_range, 'weights':weights_options}

KNN_CV = RandomizedSearchCV(KNN,params,n_iter=20,cv=5,verbose = False,scoring='roc_auc', n_jobs=-1)
## Fitting the model. 
KNN_CV.fit(X_train_Scaled, y_train)
print('Best params: ')
print(KNN_CV.best_params_) 
print('Best score: ')
print(KNN_CV.best_score_)

In [ ]:
y_pred= KNN_CV.predict_proba(X_test_Scaled)[:, 1]
y_pred_train= KNN_CV.predict_proba(X_train_Scaled)[:, 1]


In [ ]:
Plot_Roc_Curve('KNN')
Plot_Precision_Recall_Curve('KNN')

## Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

SVC = SVC()

params={'C': np.logspace(-3,3,10),'gamma': np.logspace(-2,2,5), 'kernel':['linear','rbf']}
SVC_CV=RandomizedSearchCV(SVC, params,n_iter=10,cv=3,verbose = False,scoring='roc_auc', n_jobs=-1)

SVC_CV.fit(X_train_Scaled, y_train)

print('Best params: ')
print(SVC_CV.best_params_) 
print('Best score: ')
print(SVC_CV.best_score_)

In [ ]:
y_pred= SVC_CV.predict_proba(X_test_Scaled)[:, 1]
y_pred_train= SVC_CV.predict_proba(X_train_Scaled)[:, 1]


In [ ]:
Plot_Roc_Curve('Support Vector Machine')
Plot_Precision_Recall_Curve('Support Vector Machine')

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=42)
params = {'n_estimators':[50,100,200],
          'max_features' : ['sqrt','log2'],
          'criterion': ['entropy', 'gini'],
          'min_samples_split': [2, 3, 5],
          'min_samples_leaf': [3, 4, 5, 6, 7],
          'max_depth': [7,8,9,10,11]
         }
RFC_CV = RandomizedSearchCV (RFC,params,n_iter=20,cv=5,verbose = False,scoring='roc_auc', n_jobs=-1)
RFC_CV.fit(X_train_Scaled,y_train)
print('Best params: ')
print(RFC_CV.best_params_) 
print('Best score: ')
print(RFC_CV.best_score_)

In [ ]:
y_pred=RFC_CV.predict_proba(X_test_Scaled)[:, 1]
y_pred_train=RFC_CV.predict_proba(X_train_Scaled)[:, 1]

In [ ]:
Plot_Roc_Curve('Random Forest Classifier')
Plot_Precision_Recall_Curve('Random Forest Classifier')

As a simple method to see which variables are the most relevant, we can look at the feature importances of the random forest. Given the correlations we saw in the exploratory data analysis, we should expect that the most important features are the most correlated to the target. We may use these feature importances as a method of dimensionality reduction in future work.

In [ ]:
# Extract feature importances
Features = list(X_train.columns)
feature_importance_values = RFC_CV.best_estimator_.feature_importances_
feature_importances_RFC = pd.DataFrame({'feature': Features, 'importance': feature_importance_values})

In [ ]:
# Show the feature importances for the default features
feature_importances_RFC_sorted = plot_feature_importances(feature_importances_RFC)

## Gradient Boost Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GBC= GradientBoostingClassifier()

params = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[100,150,200]
    }
GBC_CV = RandomizedSearchCV(GBC,params,cv=5,n_iter=10,verbose = False,scoring='roc_auc', n_jobs=-1)
GBC_CV.fit(X_train_Scaled,y_train)
print('Best params: ')
print(GBC_CV.best_params_) 
print('Best score: ')
print(GBC_CV.best_score_)


In [ ]:
y_pred= GBC_CV.predict_proba(X_test_Scaled)[:, 1]
y_pred_train= GBC_CV.predict_proba(X_train_Scaled)[:, 1]

In [ ]:
Plot_Roc_Curve('Gradient Boosting Classifier')
Plot_Precision_Recall_Curve('Gradient Boosting Classifier')

In [ ]:
# Extract feature importances
feature_importance_values = GBC_CV.best_estimator_.feature_importances_
feature_importances_GBC = pd.DataFrame({'feature': Features, 'importance': feature_importance_values})

In [ ]:
# Show the feature importances for the default features
feature_importances_GBC_sorted = plot_feature_importances(feature_importances_GBC)

## XGBoost Classifier

In [ ]:
from xgboost.sklearn import XGBClassifier
XGBC = XGBClassifier(objective = 'binary:logistic')


params = {
     'silent': [False],
     'max_depth': [2, 3, 4, 5],
     'learning_rate': [0.001, 0.01, 0.1, 0.15],
     'subsample': [0.7, 0.8, 0.9, 1.0],
     'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
     'colsample_bylevel': [0.7, 0.8, 0.9, 1.0],
     'min_child_weight': [0.5, 1.0, 3.0],
     'gamma': [0, 0.25, 0.5, 1.0],
     'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
     'n_estimators': [50, 100, 150],
     'scale_pos_weight': [1, 1.5, 2],
     'max_delta_step': [1, 2, 3]
 }
 
XGBC_CV = RandomizedSearchCV(XGBC,params,n_iter=20,cv=5,verbose = False,scoring='roc_auc', n_jobs=-1)
XGBC_CV.fit(X_train_Scaled,y_train)
print('Best params: ')
print(XGBC_CV.best_params_) 
print('Best score: ')
print(XGBC_CV.best_score_)


In [ ]:
y_pred= XGBC_CV.predict_proba(X_test_Scaled)[:, 1]
y_pred_train= XGBC_CV.predict_proba(X_train_Scaled)[:, 1]

In [ ]:
Plot_Roc_Curve('Extreme Gradient Boosting Classifier')
Plot_Precision_Recall_Curve('Extreme Gradient Boosting Classifier')

In [ ]:
# Extract feature importances
feature_importance_values = XGBC_CV.best_estimator_.feature_importances_
feature_importances_XGBC = pd.DataFrame({'feature': Features, 'importance': feature_importance_values})

In [ ]:
# Show the feature importances for the default features
feature_importances_XGBC_sorted = plot_feature_importances(feature_importances_XGBC)

## Customer Segmentation

In [ ]:
# Scaling Features
from sklearn.preprocessing import StandardScaler
features_scaled = StandardScaler().fit_transform(features)

In [ ]:
# Choose the number of cluster based on silhouette score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
score = []
x = list(range(2, 9))

for n_clusters in x:
    kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=40)
    kmeans.fit(features_scaled)
    clusters = kmeans.predict(features_scaled)
    silhouette_avg = silhouette_score(features_scaled, clusters)
    score.append(silhouette_avg)

In [ ]:
# Plot the evolution of the silhouette score

plt.figure(figsize=(20,16))
plt.plot(x, score)
plt.title("Evolution of the Silhouette Score")

In [ ]:
from sklearn.cluster import KMeans

n_clusters = 3
kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=30, random_state=0)
proj = kmeans.fit_transform(features_scaled)
clusters = kmeans.predict(features_scaled)

plt.figure(figsize=(10,10))
plt.scatter(proj[:,0], proj[:,1], c=clusters)
plt.title("Visualising the clusters", fontsize="20")

In [ ]:
# Visualization of the clustering with TSNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
proj = tsne.fit_transform(features_scaled)

plt.figure(figsize=(10,10))
plt.scatter(proj[:,0], proj[:,1], c=clusters)
plt.title("Visualization of the clustering with TSNE", fontsize="20")

In [ ]:
# Plot The number of customers by cluster
plt.figure(figsize = (20,8))
n, bins, patches = plt.hist(clusters, bins=3)
plt.xlabel("Cluster")
plt.title("Number of customers per cluster")
plt.xticks([rect.get_x()+ rect.get_width() / 2 for rect in patches], ["Cluster {}".format(x) for x in range(3)])
for rect in patches:
    y_value = rect.get_height()
    x_value = rect.get_x() + rect.get_width() / 2
    space = 5
    va = 'bottom'
    label = str(int(y_value))
    plt.annotate(label,(x_value, y_value),xytext=(0, space),textcoords="offset points",ha='center',va=va)

In [ ]:
# Distribution of the mostimportant features by cluster
features['clusters']=clusters
features['max_user_lifetime']=features['max_user_lifetime']/365

temp_list = list(feature_importances_RFC_sorted.iloc[:20, 1])
temp_list.append("clusters")

temp = features.loc[:, temp_list]
temp.head()

In [ ]:
import matplotlib.pyplot as plt
i=0
plt.figure(figsize=(40,120))
for col in temp.iloc[:,:-1]:
    plt.subplot((temp.shape[1]-1)/2+1,2,i+1)
    ax= sns.barplot(x='clusters',y= col ,data=temp,palette="rocket",linewidth= 2  )
    plt.title("Distribution of "+ col + " per cluster", fontsize = 25)
    plt.ylabel(col, fontsize = 20)
    plt.xlabel("Clusters",fontsize = 20)
    show_value_on_bar(ax)       
    i=i+1
    